## Bokeh Chloropleth
- Using .json file created from ESRI ABS shapefiles

In [1]:
shape_path = "/users/danielcorcoran/desktop/lga_esri_16/lga16vic_simplified/lga_vic_16_simplified.shp"

In [2]:
import fiona 
shp = fiona.open(shape_path)

In [3]:
# Extract features from shapefile
district_name = [ feat["properties"]["LGA_NAME16"] for feat in shp if feat['geometry'] is not None]
district_area = [ feat["properties"]["AREASQKM16"] for feat in shp if feat['geometry'] is not None]
district_x = [ [x[0] for x in feat["geometry"]["coordinates"][0]] for feat in shp if feat['geometry'] is not None]
district_y = [ [y[1] for y in feat["geometry"]["coordinates"][0]] for feat in shp if feat['geometry'] is not None]

In [31]:
import bokeh
from bokeh.plotting import figure, show, output_file
import numpy
from bokeh.models import HoverTool

TOOLS = "pan,wheel_zoom,reset,hover,save"

possible_colours = ['#ffffcc','#a1dab4','#41b6c4','#2c7fb8','#253494']
colours = [possible_colours[numpy.random.randint(0,4)] for n in range(len(district_x))]
p = figure(
    
    title="Local Government Areas (2016) Bokeh Plot",
    toolbar_location="right",
    
    plot_width=1100,
    plot_height=800,
    tools = TOOLS
    
#     x_axis_location=None,
#     y_axis_location=None
)

p.patches(
    district_x, 
    district_y,
    fill_alpha=1, 
    line_color="white", 
    line_width=1,
    fill_color = colours
    )
hover = p.select_one(HoverTool)
hover.point_policy = "follow_mouse"
hover.tooltips = [('this','@district_x'), ('Longitude, Latitude', '$x,$y')]

#output_file("choropleth.html", title="choropleth.py example")

show(p)

In [ ]:
for i,name in enumerate(shp):
    print(i)
    print(name['properties']['LGA_NAME16'])

In [ ]:
from bokeh.io import show, output_notebook, output_file
from bokeh.models import (
    GeoJSONDataSource,
    HoverTool,
    LinearColorMapper
)
from bokeh.plotting import figure
from bokeh.palettes import Viridis6

with open('/users/danielcorcoran/desktop/LGA16_SIMPLIFIED.geojson', 'r') as f:
    geo_source = GeoJSONDataSource(geojson=f.read())

color_mapper = LinearColorMapper(palette=Viridis6)

TOOLS = "pan,wheel_zoom,box_zoom,reset,hover,save"

p = figure(title="Title", tools=TOOLS, x_axis_location=None, y_axis_location=None, width=1200, height=800)
p.grid.grid_line_color = None

p.patches('xs', 'ys', fill_alpha=0.7, fill_color={'field': 'objectid', 'transform': color_mapper}, 
          line_color='white', line_width=0.5, source=geo_source)


# hover = p.select_one(HoverTool)
# hover.point_policy = "follow_mouse"
# hover.tooltips = [("Provincia:", "@provincia")]

# output_file("PBIar.html", title="Testing islands in bokeh")

show(p)